In [1]:
import os
import cv2
import face_recognition
import numpy as np
import tkinter as tk
from tkinter import Label, Button, messagebox
from PIL import Image, ImageTk
import pandas as pd
from datetime import datetime
import pickle

with open("encodings.pkl", "rb") as f:
    encodelistdone, persons_names = pickle.load(f)

attendance = {}
current_name = None

window = tk.Tk()
window.title("Face Attendance System")
window.geometry("850x650")
window.configure(bg="#e6f2ff")

title_label = Label(window, text="Face Attendance System ", font=("Helvetica", 18, "bold"),
                    fg="#003366", bg="#e6f2ff")
title_label.pack(pady=15)

video_label = Label(window, bg="#e6f2ff")
video_label.pack()

name_label = Label(window, text="No Face Detected", font=("Helvetica", 14),
                   fg="#003366", bg="#e6f2ff")
name_label.pack(pady=10)

def mark_attendance():
    global current_name, name_label

    if not current_name:
        return

    today = datetime.now().strftime("%Y-%m-%d")
    desktop_path = os.path.join(os.path.expanduser("~"), "Desktop")
    filename = os.path.join(desktop_path, "FaceAttendance_Project.xlsx")

    if os.path.exists(filename):
        df = pd.read_excel(filename)
    else:
        df = pd.DataFrame({"Name": persons_names})

    for name in persons_names:
        if name not in df["Name"].values:
            df = pd.concat([df, pd.DataFrame({"Name": [name]})], ignore_index=True)

    if today not in df.columns:
        df[today] = pd.Series([pd.NA] * len(df), dtype="object")

    if current_name not in df["Name"].values:
        df = pd.concat([df, pd.DataFrame({"Name": [current_name]})], ignore_index=True)

    index = df[df["Name"] == current_name].index[0]
    current_value = df.at[index, today]

    if current_value == True:
        name_label.config(text=f"{current_name}  Already Marked")
        messagebox.showinfo("Already Marked", f"{current_name} is already marked on {today}.")
    else:
        df.at[index, today] = True
        df.to_excel(filename, index=False)
        name_label.config(text=f"{current_name}  Marked")
        messagebox.showinfo("Attendance Recorded", f"{current_name} marked present successfully.")

confirm_button = Button(window, text="Confirm Attendance",
                        font=("Helvetica", 13, "bold"),
                        bg="#28a745", fg="white", padx=20, pady=10,
                        command=mark_attendance)
confirm_button.pack(pady=15)

cap = cv2.VideoCapture(0)

def update_frame():
    global current_name
    ret, img = cap.read()
    if not ret:
        return

    small_frame = cv2.resize(img, (0, 0), fx=0.25, fy=0.25)
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    faces_current_frame = face_recognition.face_locations(rgb_small_frame)
    encodes_current_frame = face_recognition.face_encodings(rgb_small_frame, faces_current_frame)

    name = "No Face Detected"
    for encodeface, facelocation in zip(encodes_current_frame, faces_current_frame):
        matches = face_recognition.compare_faces(encodelistdone, encodeface)
        face_distance = face_recognition.face_distance(encodelistdone, encodeface)
        matchIndex = np.argmin(face_distance)

        if matches[matchIndex]:
            name = persons_names[matchIndex].capitalize()
            current_name = name
        else:
            current_name = None

    name_label.config(text=name)

    rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_pil = Image.fromarray(rgb_image)
    imgtk = ImageTk.PhotoImage(image=img_pil)
    video_label.imgtk = imgtk
    video_label.configure(image=imgtk)

    window.after(10, update_frame)

update_frame()
window.mainloop()
cap.release()
cv2.destroyAllWindows()
